In [3]:
#pip install pandas

In [2]:
import pandas, json
from datetime import date

#function for getting data from the woolworths api
def download(url = 'http://www.wikipedia.org/',
             target_filename = 'download',
             filename_extension = 'html',
             save_file = True,
             char_set = 'UTF-8',
             lying = False,
             got_the_message = True):

    # Import the function for opening online documents and
    # the class for creating requests
    from urllib.request import urlopen, Request

    # Import an exception raised when a web server denies access
    # to a document
    from urllib.error import HTTPError

    # Open the web document for reading
    try:
        if lying:
            # Pretend to be something other than a Python
            # script (NOT RELIABLE OR RECOMMENDED!)
            request = Request(url)
            request.add_header('User-Agent', 'Mozilla/5.0')
            if not got_the_message:
                print("Warning - Request does not reveal client's true identity.")
                print("          This is both unreliable and unethical!")
                print("          Proceed at your own risk!\n")
        else:
            # Behave ethically
            request = url
        web_page = urlopen(request)
    except ValueError:
        print("Download error - Cannot find document at URL '" + url + "'\n")
        return None
    except HTTPError:
        print("Download error - Access denied to document at URL '" + url + "'\n")
        return None
    except Exception as message:
        print("Download error - Something went wrong when trying to download " + \
              "the document at URL '" + url + "'")
        print("Error message was:", message, "\n")
        return None

    # Read the contents as a character string
    try:
        web_page_contents = web_page.read().decode(char_set)
    except UnicodeDecodeError:
        print("Download error - Unable to decode document from URL '" + \
              url + "' as '" + char_set + "' characters\n")
        return None
    except Exception as message:
        print("Download error - Something went wrong when trying to decode " + \
              "the document from URL '" + url + "'")
        print("Error message was:", message, "\n")
        return None

    # Optionally write the contents to a local text file
    # (overwriting the file if it already exists!)
    if save_file:
        try:
            text_file = open(target_filename + '.' + filename_extension,
                             'w', encoding = char_set)
            text_file.write(web_page_contents)
            text_file.close()
        except Exception as message:
            print("Download error - Unable to write to file '" + \
                  target_filename + "'")
            print("Error message was:", message, "\n")

    # Return the downloaded document to the caller
    return web_page_contents

In [25]:
#reading in dataframe with previous prices
df = pandas.read_csv('out.csv', parse_dates=['Date'], usecols=['Date', 'Name', 'Category', 'Price'])
df

,Date,Name,Category,Price
0,2023-02-09,Fresh Strawberry 250G,Fruit,3.00
1,2023-02-09,White Seedless Grapes Bunch Each,Fruit,6.56
2,2023-02-09,Fresh Blueberries 125G,Fruit,3.50
3,2023-02-09,Driscoll's Fresh Raspberry 125G,Fruit,2.90
4,2023-02-09,Cavendish Bananas Each,Fruit,0.81
...,...,...,...,...
79,2023-02-09,Weis Ice Cream Mango & Cream 4 Pack,Frozen,8.50
80,2023-02-09,"Bulla Splits Multipack Raspberry, Mango & Lemo...",Frozen,8.00
81,2023-02-09,Streets Blue Ribbon Ice Cream Vanilla 2L,Frozen,5.75
82,2023-02-09,Connoisseur Ice Cream Vanilla Caramel Brownie 1L,Frozen,12.00


In [4]:
fruit = ['144607', '138801', '169792', '165262', '133211', '120384', '130935', '245902', '259514', '310966', '306343', '149885', '141336']
veg = ['524336', '54900', '149620', '147603', '120080', '135369', '134681', '137102', '208895', '144497', '144329', '147071', '169067', '141496', '187314', '162931', '138082', '170127', '713429', 
    '134072', '149374', '132538', '206036']
meat = ['321141', '764465', '577863', '77069', '675318']
fridge = ['208064', '310675', '359118', '194348', '76651', '44317', '96888', '75073', '769558', '43562']
pantry = ['4516', '144283', '45813', '5743', '37679', '96253', '829004', '490219', '813868', '813865', '774157', '94124', '728628', '38921', '280512', '202915', '199725', '192985', '829281', '262206', 
        '713816', '3805', '130502', '573301']
frozen = ['260007', '260010', '754181', '46280', '178953', '140646', '40568', '699140', '180294']
items = [fruit, veg, meat, fridge, pantry, frozen]
categories = ["Fruit", 'Vegatable', 'Meat', 'Fridge','Pantry', 'Frozen']

#getting all the current prices from today
category_counter = 0
for category in items:
    for product in category:
        url = 'https://www.woolworths.com.au/api/v3/ui/schemaorg/product/' + product
        data = json.loads(download(url, lying=True, save_file=False))
        df = pandas.concat([df, pandas.DataFrame([[date.today(), data['name'], categories[category_counter], data['offers']['price']]],
                    columns=['Date', 'Name', 'Category', 'Price'])])
    category_counter = category_counter + 1

df

,Date,Name,Category,Price
0,2023-02-09 00:00:00,Fresh Strawberry 250G,Fruit,3.00
0,2023-02-09 00:00:00,White Seedless Grapes Bunch Each,Fruit,6.56
0,2023-02-09 00:00:00,Fresh Blueberries 125G,Fruit,3.50
0,2023-02-09 00:00:00,Driscoll's Fresh Raspberry 125G,Fruit,2.90
0,2023-02-09,Cavendish Bananas Each,Fruit,0.81
...,...,...,...,...
0,2023-02-09,Weis Ice Cream Mango & Cream 4 Pack,Frozen,8.50
0,2023-02-09,"Bulla Splits Multipack Raspberry, Mango & Lemo...",Frozen,8.00
0,2023-02-09,Streets Blue Ribbon Ice Cream Vanilla 2L,Frozen,5.75
0,2023-02-09,Connoisseur Ice Cream Vanilla Caramel Brownie 1L,Frozen,12.00


In [23]:
#saves the current dataframe and reloads it so that the dates are parsed
df.to_csv('out.csv')
df = pandas.read_csv('out.csv', parse_dates=['Date'], usecols=['Date', 'Name', 'Category', 'Price'])

In [24]:
df

,Date,Name,Category,Price
0,2023-02-09,Fresh Strawberry 250G,Fruit,3.00
1,2023-02-09,White Seedless Grapes Bunch Each,Fruit,6.56
2,2023-02-09,Fresh Blueberries 125G,Fruit,3.50
3,2023-02-09,Driscoll's Fresh Raspberry 125G,Fruit,2.90
4,2023-02-09,Cavendish Bananas Each,Fruit,0.81
...,...,...,...,...
79,2023-02-09,Weis Ice Cream Mango & Cream 4 Pack,Frozen,8.50
80,2023-02-09,"Bulla Splits Multipack Raspberry, Mango & Lemo...",Frozen,8.00
81,2023-02-09,Streets Blue Ribbon Ice Cream Vanilla 2L,Frozen,5.75
82,2023-02-09,Connoisseur Ice Cream Vanilla Caramel Brownie 1L,Frozen,12.00


In [ ]:
#TODO: DO CALCULATIONS, ADD CHARTS ETC.